# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
import os
import pandas as pd

In [3]:
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="lol_match_predictor_automl")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-136182
Azure region: southcentralus
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-136182


In [4]:
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "automl_cluster"

# Create a provisioning configuration using the Standard_D2_V2 VM and capping it at 4 nodes.
compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes=4)
# Create compute cluster.
#cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target = ComputeTarget(workspace=ws, name="cluster")

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
data = pd.read_csv("https://oracleselixir-downloadable-match-data.s3-us-west-2.amazonaws.com/2020_LoL_esports_match_data_from_OraclesElixir_20210126.csv")

# This dataset requires cleaning to only get the features we want.
columns = ['side', 'position', 'result', 'golddiffat10', 'xpdiffat10', 'golddiffat15', 'xpdiffat15']

data = pd.DataFrame(data, columns=columns)

# This dataset has data for both sides in each match. We only need to look at matches from one side, so we take only Blue side data.

data = data.loc[data['side'] == 'Blue']

# This dataset also has data for teams as well as individual players. We only need the data for teams.

data = data[data.position.eq('team')]

# Side no longer needed so we can delete this field.
data.drop(['side'], inplace=True, axis=1, errors='ignore')
# We also no longer need positions.
data.drop(['position'], inplace=True, axis=1, errors='ignore')

# This dataset is equivalent to the output of the above operations.
ds = TabularDatasetFactory.from_delimited_files(path="https://raw.githubusercontent.com/ncharchenko/Azure-ML-Engineer-Nanodegree-Capstone/master/OraclesElixir_automl_data.csv")

# Show data after cleaning.
data.head()

,result,golddiffat10,xpdiffat10,golddiffat15,xpdiffat15
10,1,117.00,-137.00,-825.00,-1665.00
22,0,-2014.00,-2149.00,-5484.00,-4833.00
34,1,682.00,615.00,2566.00,2097.00
46,0,-1579.00,-858.00,-2471.00,-3064.00
58,0,-747.00,-313.00,877.00,205.00


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
# TODO: Put your automl settings here
automl_settings = {}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    enable_onnx_compatible_models=True,
    compute_target=compute_target,
    task="classification",
    primary_metric="accuracy",
    training_data=ds,
    validation_size=0.2,
    label_column_name="result")

In [7]:
# TODO: Submit your experiment
remote_run = exp.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
best_automl, automl_model = remote_run.get_output()
best_automl, onnx_model = remote_run.get_output(return_onnx_model=True)
os.makedirs("outputs", exist_ok=True)

In [ ]:
#TODO: Save the best model
OnnxConverter.save_onnx_model(onnx_model,"./outputs/automl_model.onnx")

print(best_automl.get_tags())

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service